In [1]:
%matplotlib qt
import matplotlib.pyplot as plt

import os.path as op
import numpy as np
import mne

from scipy.signal import savgol_filter

mne.set_log_level('WARNING')

subjects_dir = op.expanduser("~/data/pic-name-data-bids/MRI/")
res_dir = op.expanduser("~/research/results/picname")

In [2]:
database = {"Group-3": ["sub-%02d" % i for i in [2, 8, 9, 12]],
            "Group-2": ["sub-%02d" % i for i in [1, 6, 10, 11, 15]],
            "Group-1": ["sub-%02d" % i for i in [3, 4, 5, 13, 14]],
            "All":  ["sub-%02d" % i for i in range(1, 16) if i not in [7]],
           }
ROIs = ["LO", "Wernicke", "Broca", "Fusiform"]
conditions = ["Group-1", "Group-2", "Group-3", "All"]

In [3]:
def get_roi_labels(subject, subjects_dir, name, dominant='left'):
    if name == "LO":
        labels = mne.read_labels_from_annot(subject, parc='aparc', subjects_dir=subjects_dir)
        ROI = ['lateraloccipital-lh', 'lateraloccipital-rh']
        labels_roi = []
        for lbl in labels:
            if lbl.name in ROI:
                print(lbl.name)
                labels_roi.append(lbl)
        label = labels_roi[0]
        for i in range(1, len(labels_roi)):
            label = label + labels_roi[i]
        return label
    elif name == "Wernicke":
        labels = mne.read_labels_from_annot(subject, parc='PALS_B12_Brodmann', subjects_dir=subjects_dir)
        if dominant == 'left':
            ROI = ['Brodmann.22-lh']
        elif dominant == "right":
            ROI = ['Brodmann.22-rh']
        else:
            raise ValueError("$dominant can either be 'left' or 'right'. Check input value.")
        labels_roi = []
        for lbl in labels:
            if lbl.name in ROI:
                print(lbl.name)
                labels_roi.append(lbl)
        label = labels_roi[0]
        for i in range(1, len(labels_roi)):
            label = label + labels_roi[i]
        return label
    elif name == "Broca":
        labels = mne.read_labels_from_annot(subject, parc='PALS_B12_Brodmann', subjects_dir=subjects_dir)
        if dominant == 'left':
            ROI = ['Brodmann.44-lh', 'Brodmann.45-lh']
        elif dominant == "right":
            ROI = ['Brodmann.44-rh', 'Brodmann.45-rh']
        else:
            raise ValueError("$dominant can either be 'left' or 'right'. Check input value.")
        labels_roi = []
        for lbl in labels:
            if lbl.name in ROI:
                print(lbl.name)
                labels_roi.append(lbl)
        label = labels_roi[0]
        for i in range(1, len(labels_roi)):
            label = label + labels_roi[i]
        return label
    elif name == "Fusiform":
        labels = mne.read_labels_from_annot(subject, parc='aparc', subjects_dir=subjects_dir)
        if dominant == 'left':
            ROI = ['fusiform-lh']
        elif dominant == "right":
            ROI = ['fusiform-rh']
        else:
            raise ValueError("$dominant can either be 'left' or 'right'. Check input value.")
        labels_roi = []
        for lbl in labels:
            if lbl.name in ROI:
                print(lbl.name)
                labels_roi.append(lbl)
        label = labels_roi[0]
        for i in range(1, len(labels_roi)):
            label = label + labels_roi[i]
        return label
    else:
        raise ValueError('No such label available. Check input value.')
    return None

In [4]:
def group_analysis_cond_label(cond, roi):
    """
    Find evoked response for the given 'cond' of subjects averaged over the 'roi' sources.
    """
    # Find subjects for given condition and print them
    
    subjects = database.get(cond, None)
    print(cond, subjects)
            
    # Get STCS for all subjects depending on the dominant brain hemispheres
    stc_fnames = []
    N_avg = []
    for subject in subjects:
        stc_fnames.append(op.join(res_dir, subject + '/dspm_%s' % subject + '-lh.stc'))
        fname = op.join(res_dir, subject, subject+'-ave.fif')
        ev = mne.read_evokeds(fname)
        N_avg.append(ev[0].nave)

    stcs = [mne.read_source_estimate(stc_fname) for stc_fname in stc_fnames]
    times = stcs[0].times
    
    stc_label_mean = []
    for isub, stc in enumerate(stcs):
        subject = subjects[isub]
        label = get_roi_labels(subject, subjects_dir, roi, 'left')
        src = mne.setup_source_space(subject, spacing='oct6', subjects_dir=subjects_dir,
                            add_dist=False)
        label_ts = mne.extract_label_time_course(stc, label, src, mode='pca_flip',
                                                return_generator=True)
        label_ts_smooth = savgol_filter(label_ts, 251, 3) # window size 251, polynomial order 3
        stc_label_mean.extend(label_ts_smooth)
    average_stc_label_mean = np.average(stc_label_mean, axis=0, weights=N_avg)
    
    return times, average_stc_label_mean

In [9]:
def group_analysis_evoked():
    """
    Perform groupwise evoked response analysis for all subject conditions and regions of interest.
    """
    average_stc_label = {}
    for cond in conditions:
        for roi in ROIs:
            times, average_stc_label[(cond, roi)] = group_analysis_cond_label(cond, roi)

    # Get Evoked Responses for all conditions in each ROI
    times = 1e3 * times
    style = ['m-.', 'r-', 'k--']
    ticks = np.arange(-0.5, 1.0, 0.05)
    for roi in ROIs:
        fig = plt.figure()
        ax = fig.add_subplot(1, 1, 1)
        h = [None] * len(conditions)
        for i, cond in enumerate(conditions):
            if cond != "All":
                h[i], = plt.plot(times, average_stc_label[(cond, roi)], style[i], linewidth=1.5)
        plt.xlim(times[0], times[-1])
        plt.legend((h[0], h[1], h[2]), conditions[:-1])
        plt.xlabel('Time (ms)')
        plt.ylabel('dSPM value')
        ax.set_xticks(ticks, minor=True)
        ax.grid(which='both')
        plt.title("Evoked response in " + roi)
        plt.savefig(op.join(res_dir, 'evoked_label_' + roi + '.pdf'))
        
#     # Get Evoked Responses in LO averaged over all subjects
#     fig = plt.figure()
#     ax = fig.add_subplot(1, 1, 1)
#     h = plt.plot(times, average_stc_label[("all", "LO")], 'r', linewidth=3)
#     plt.xlim(times[0], times[-1])
#     plt.xlabel('Time (ms)')
#     plt.ylabel('dSPM value')
#     ax.set_xticks(ticks, minor=True)
#     ax.grid(which='both')
#     plt.title("Evoked response in LO averaged over all subjects")
#     plt.savefig(op.join(res_dir, 'evoked_label_all_LO.pdf'))

    # Get Evoked Responses for all 3 ROIs in each condition
    for cond in conditions:
        fig = plt.figure()
        ax = fig.add_subplot(1, 1, 1)
        h = [None] * 4
        for j, roi in enumerate(ROIs):
            h[j], = plt.plot(times, average_stc_label[(cond, roi)])
        plt.xlim(times[0], times[-1])
        plt.legend((h[0], h[1], h[2], h[3]), ROIs)
        plt.xlabel('Time (ms)')
        plt.ylabel('dSPM value')
        ax.set_xticks(ticks, minor=True)
        ax.grid(which='both')
        plt.title("Evoked response in " + cond)
        plt.savefig(op.join(res_dir, 'evoked_label_' + cond + '.pdf'))
    return None

In [11]:
def group_analysis_cortical(cond):
    """
    Get cortical activations averaged over any of the 2 dominant-hemisphere subjects.
    """
    # Find subjects for given condition and print them
    subjects = database.get(cond, None)
    print(cond, subjects)
        
    # Get STCS for all subjects depending on the dominant brain hemispheres
    stc_fnames = []
    for subject in subjects:
        stc_fnames.append(op.join(res_dir, subject + '/dspm_%s' % subject + '-lh.stc'))

    stcs = [mne.read_source_estimate(stc_fname) for stc_fname in stc_fnames]

    # Average all source time courses
    average_stc = np.mean(stcs)
    average_stc_bin = average_stc.bin(width=0.25, tstart=0, tstop=1, func=np.mean)

    # Find peak value and time instant of average response
    vertno_peak, t_peak = average_stc_bin.get_peak()
    print('Peak dSPM at', t_peak, 's')

    # Draw average cortical activation image and movie
    peak_times = [.125, .375, .625, .875]
    surfer_kwargs = dict(subject="sub-00", hemi='lh', surface='pial', subjects_dir=subjects_dir,
            clim=dict(kind='value', lims=[2, 3, 5]), views=['lateral', 'medial', 'ventral'],
            time_unit='s', size=(800, 800), smoothing_steps=20, time_viewer=False)
    
    for tp in peak_times:
        surfer_kwargs['initial_time'] = tp
        brain = average_stc_bin.plot(**surfer_kwargs)
        # Save image
        dspm_fname = op.join(res_dir, 'dspm_sub-00_' + cond + '_%02f.png' % tp)
        brain.save_image(dspm_fname)
        brain.close()

    # Save movie
    surfer_kwargs['clim'] = dict(kind='value', lims=[3, 4, 6])
    brain = average_stc.plot(**surfer_kwargs)
    dspm_movie_fname = op.join(res_dir, 'dspm_movie_sub-00_' + cond + '.mov')
    brain.save_movie(dspm_movie_fname, tmin=0.05, tmax=0.55, interpolation='linear',
                    time_dilation=20, framerate=10, time_viewer=True)
    brain.close()
    return None

In [10]:
group_analysis_evoked()

Group-1 ['sub-03', 'sub-04', 'sub-05', 'sub-13', 'sub-14']
lateraloccipital-lh
lateraloccipital-rh
lateraloccipital-lh
lateraloccipital-rh
lateraloccipital-lh
lateraloccipital-rh
lateraloccipital-lh
lateraloccipital-rh
lateraloccipital-lh
lateraloccipital-rh
Group-1 ['sub-03', 'sub-04', 'sub-05', 'sub-13', 'sub-14']
Brodmann.22-lh
Brodmann.22-lh
Brodmann.22-lh
Brodmann.22-lh
Brodmann.22-lh
Group-1 ['sub-03', 'sub-04', 'sub-05', 'sub-13', 'sub-14']
Brodmann.44-lh
Brodmann.45-lh
Brodmann.44-lh
Brodmann.45-lh
Brodmann.44-lh
Brodmann.45-lh
Brodmann.44-lh
Brodmann.45-lh
Brodmann.44-lh
Brodmann.45-lh
Group-1 ['sub-03', 'sub-04', 'sub-05', 'sub-13', 'sub-14']
fusiform-lh
fusiform-lh
fusiform-lh
fusiform-lh
fusiform-lh
Group-2 ['sub-01', 'sub-06', 'sub-10', 'sub-11', 'sub-15']
lateraloccipital-lh
lateraloccipital-rh
lateraloccipital-lh
lateraloccipital-rh
lateraloccipital-lh
lateraloccipital-rh
lateraloccipital-lh
lateraloccipital-rh
lateraloccipital-lh
lateraloccipital-rh
Group-2 ['sub-01', '

In [16]:
group_analysis_cortical("All")

All ['sub-01', 'sub-02', 'sub-03', 'sub-04', 'sub-05', 'sub-06', 'sub-08', 'sub-09', 'sub-10', 'sub-11', 'sub-12', 'sub-13', 'sub-14', 'sub-15']
Peak dSPM at 0.375 s
